In [1]:
import metview as mv
import datetime
from ecmwf.opendata import Client

# Proses data ke CSV

# 1. Gabungkan semua file jadi satu dataset
Sementara ini baru 3 hari data, tapi nanti kalau sudah 1 bulan bisa langsung gabung semua file jadi satu dataset.

In [13]:
import xarray as xr
from glob import glob

files = sorted(glob("medium-2t-wind*.grib2"))

datasets = []
for f in files:
    try:
        ds = xr.open_dataset(
            f,
            engine="cfgrib",
            backend_kwargs={
                "filter_by_keys": {"shortName": "2t"}  # hanya ambil t2m
            }
        )
        datasets.append(ds)
    except Exception as e:
        print(f"Skip {f}: {e}")

# Gabungkan sepanjang dimensi time
ds_all = xr.concat(datasets, dim="time")
print(ds_all)

/home/prema/miniconda3/envs/notebooks/lib/python3.13/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/home/prema/miniconda3/envs/notebooks/lib/python3.13/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it

<xarray.Dataset> Size: 50MB
Dimensions:            (time: 3, step: 4, latitude: 721, longitude: 1440)
Coordinates:
  * time               (time) datetime64[ns] 24B 2025-09-02 ... 2025-09-04
  * step               (step) timedelta64[ns] 32B 00:00:00 06:00:00 ... 18:00:00
    heightAboveGround  float64 8B 2.0
  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude          (longitude) float64 12kB -180.0 -179.8 ... 179.5 179.8
    valid_time         (time, step) datetime64[ns] 96B 2025-09-02 ... 2025-09...
Data variables:
    t2m                (time, step, latitude, longitude) float32 50MB 268.6 ....
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-05T12:14 GRIB t

/tmp/ipykernel_14662/277129378.py:21: FutureWarning: In a future version of xarray the default value for coords will change from coords='different' to coords='minimal'. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set coords explicitly.
  ds_all = xr.concat(datasets, dim="time")


# 2. Ambil variabel (contoh suhu 2m)

In [16]:
# Ambil variabel (contoh suhu 2m)
t2m = ds_all['t2m']

# Konversi ke Celsius
t2m_celsius = t2m - 273.15

# 3. Ekstrak data ke NumPy array

In [17]:
import numpy as np

# Bentuk data: (time, step, lat, lon)
arr = t2m_celsius.values

print("Shape data:", arr.shape)  # contoh: (31, 24, 721, 1440)

Shape data: (3, 4, 721, 1440)


# 4. Ekstrak data ke CSV (contoh: ambil 1 lokasi tertentu)

Biasanya ML butuh tabular dataset (time series). Kita bisa pilih 1 gridpoint (lat/lon) dan simpan ke CSV.

In [18]:
# Pilih lokasi (misalnya Tokyo: lat 35.7, lon 139.7)
lat, lon = 35.7, 139.7

# Cari grid terdekat
point = t2m_celsius.sel(latitude=lat, longitude=lon, method="nearest")

# Ubah ke dataframe
df = point.to_dataframe().reset_index()

# Simpan ke CSV
df.to_csv("tokyo_temp_last_month.csv", index=False)

print(df.head())

        time            step  heightAboveGround  latitude  longitude  \
0 2025-09-02 0 days 00:00:00                2.0     35.75     139.75   
1 2025-09-02 0 days 06:00:00                2.0     35.75     139.75   
2 2025-09-02 0 days 12:00:00                2.0     35.75     139.75   
3 2025-09-02 0 days 18:00:00                2.0     35.75     139.75   
4 2025-09-03 0 days 00:00:00                2.0     35.75     139.75   

           valid_time        t2m  
0 2025-09-02 00:00:00  32.500824  
1 2025-09-02 06:00:00  35.352875  
2 2025-09-02 12:00:00  30.665466  
3 2025-09-02 18:00:00  28.483734  
4 2025-09-03 00:00:00  32.919647  


# 5. Siapkan data untuk ML

In [20]:
import pandas as pd

# Baca CSV
df = pd.read_csv("tokyo_temp_last_month.csv")
df

,time,step,heightAboveGround,latitude,longitude,valid_time,t2m
0,2025-09-02,0 days 00:00:00,2.0,35.75,139.75,2025-09-02 00:00:00,32.500824
1,2025-09-02,0 days 06:00:00,2.0,35.75,139.75,2025-09-02 06:00:00,35.352875
2,2025-09-02,0 days 12:00:00,2.0,35.75,139.75,2025-09-02 12:00:00,30.665466
3,2025-09-02,0 days 18:00:00,2.0,35.75,139.75,2025-09-02 18:00:00,28.483734
4,2025-09-03,0 days 00:00:00,2.0,35.75,139.75,2025-09-03 00:00:00,32.919647
5,2025-09-03,0 days 06:00:00,2.0,35.75,139.75,2025-09-03 06:00:00,34.774690
6,2025-09-03,0 days 12:00:00,2.0,35.75,139.75,2025-09-03 12:00:00,30.089020
7,2025-09-03,0 days 18:00:00,2.0,35.75,139.75,2025-09-03 18:00:00,26.102448
8,2025-09-04,0 days 00:00:00,2.0,35.75,139.75,2025-09-04 00:00:00,28.676880
9,2025-09-04,0 days 06:00:00,2.0,35.75,139.75,2025-09-04 06:00:00,28.185577


In [22]:
# Buat target = suhu +24 jam
df["target"] = df["t2m"].shift(-24)

# X = fitur (suhu saat ini), y = target (besok)
X = df[["t2m"]].values
y = df["target"].values
print("X shape:", X.shape, "y shape:", y.shape)

X shape: (12, 1) y shape: (12,)
